# Making new layers and models via subclassing


## Importing Libraries

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

## Best practice: deferring weight creation until the shape of the inputs is known

Our `Linear` layer above took an `input_dim `argument that was used to compute
the shape of the weights `w` and `b` in `__init__()`:

In [2]:

class Linear(keras.layers.Layer):
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(
            shape=(input_dim, units), initializer="random_normal", trainable=True
        )
        self.b = self.add_weight(shape=(units,), initializer="zeros", trainable=True)

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


In many cases, you may not know in advance the size of your inputs, and you
would like to lazily create weights when that value becomes known, some time
after instantiating the layer.

In the Keras API, we recommend creating layer weights in the `build(self,
inputs_shape)` method of your layer. Like this:

In [3]:

class Linear(keras.layers.Layer):
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


The `__call__()` method of your layer will automatically run build the first time
it is called. You now have a layer that's lazy and thus easier to use:

In [5]:
# At instantiation, we don't know on what inputs this is going to get called
linear_layer = Linear(32)

# The layer's weights are created dynamically the first time the layer is called
x = tf.ones((2, 2))
y = linear_layer(x)
print(y)

tf.Tensor(
[[ 0.01756852  0.10476129  0.12511107 -0.1029959  -0.06927948  0.0204796
  -0.17278597  0.01521104 -0.1969185  -0.12868682  0.04628122 -0.0146234
   0.09063937 -0.04699198 -0.04217857  0.02450774 -0.14576295  0.12174433
   0.03162061 -0.05990808  0.13012084  0.08229318  0.00379721  0.02050176
  -0.09409647 -0.0253645   0.0466613  -0.02622731 -0.02764079 -0.09329426
   0.12508042  0.09328259]
 [ 0.01756852  0.10476129  0.12511107 -0.1029959  -0.06927948  0.0204796
  -0.17278597  0.01521104 -0.1969185  -0.12868682  0.04628122 -0.0146234
   0.09063937 -0.04699198 -0.04217857  0.02450774 -0.14576295  0.12174433
   0.03162061 -0.05990808  0.13012084  0.08229318  0.00379721  0.02050176
  -0.09409647 -0.0253645   0.0466613  -0.02622731 -0.02764079 -0.09329426
   0.12508042  0.09328259]], shape=(2, 32), dtype=float32)


## Layers are recursively composable

If you assign a Layer instance as an attribute of another Layer, the outer layer
will start tracking the weights of the inner layer.

We recommend creating such sublayers in the `__init__()` method (since the
sublayers will typically have a build method, they will be built when the
outer layer gets built).

In [6]:
# Let's assume we are reusing the Linear class
# with a `build` method that we defined above.


class MLPBlock(keras.layers.Layer):
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(1)

    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)



In [7]:
mlp = MLPBlock()
y = mlp(tf.ones(shape=(3, 64)))  # The first call to the `mlp` will create the weights
print("weights:", len(mlp.weights))
print("trainable weights:", len(mlp.trainable_weights))

weights: 6
trainable weights: 6
